In [53]:
from roboflow import Roboflow
import json

from roboflow import Roboflow
rf = Roboflow(api_key="6Y4RTIEZtXjfjclMhIx1")
project = rf.workspace("ki-bu8lz").project("ki-dybmq")
version = project.version(8)
dataset = version.download("coco")
                

loading Roboflow workspace...
loading Roboflow project...


In [54]:
import os
import shutil

# Das aktuelle Arbeitsverzeichnis ermitteln
dir_path = os.getcwd()

# Pfade für Trainings- und Validierungsordner definieren
input_path_train = os.path.join(dir_path, f"KI-{version.version}", "train")
input_path_valid = os.path.join(dir_path, f"KI-{version.version}", "valid")  # Rechtschreibung korrigiert: 'valid' statt 'vaild'
output_path_train = os.path.join(dir_path, "dataset_project", "images", "train") 
output_path_valid = os.path.join(dir_path, "dataset_project", "images", "valid")  # Rechtschreibung korrigiert: 'valid' statt 'vaild'

# Funktion zum Kopieren der Bilder und JSON-Dateien
def copy_files(source_folder, target_folder):
    # Erstellen des Zielordners, falls er nicht existiert
    os.makedirs(target_folder, exist_ok=True)
    
    # Liste der Dateien im Quellordner abrufen
    file_names = os.listdir(source_folder)
    
    # Kopieren der Dateien (Bilder und JSON-Dateien)
    for filename in file_names:
        source_path = os.path.join(source_folder, filename)
        target_path = os.path.join(target_folder, filename)
        
        # Wenn es sich um eine Bilddatei oder eine JSON-Datei handelt, kopieren
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.json')):
            try:
                shutil.copy(source_path, target_path)
                print(f"Datei '{filename}' wurde nach '{target_path}' kopiert.")
            except FileNotFoundError:
                print(f"Datei '{filename}' wurde nicht gefunden: {source_path}")
            except Exception as e:
                print(f"Fehler beim Kopieren von '{filename}': {e}")

# Trainingsbilder und Annotationsdateien kopieren
copy_files(input_path_train, output_path_train)

# Validierungsbilder und Annotationsdateien kopieren
copy_files(input_path_valid, output_path_valid)


Datei 'biological775_jpg.rf.1d13aeae9daa6c1048919874d8c1bb5f.jpg' wurde nach 'c:\Users\thore\Documents\GitHub\ki\dataset_project\images\train\biological775_jpg.rf.1d13aeae9daa6c1048919874d8c1bb5f.jpg' kopiert.
Datei 'biological787_jpg.rf.70df4f651b4a64c037e77101e55ddc55.jpg' wurde nach 'c:\Users\thore\Documents\GitHub\ki\dataset_project\images\train\biological787_jpg.rf.70df4f651b4a64c037e77101e55ddc55.jpg' kopiert.
Datei 'biological80_jpg.rf.eca050bc55a6feee750cbe400fe9f568.jpg' wurde nach 'c:\Users\thore\Documents\GitHub\ki\dataset_project\images\train\biological80_jpg.rf.eca050bc55a6feee750cbe400fe9f568.jpg' kopiert.
Datei 'biological826_jpg.rf.422d96721550d77bd7e8a724d87bd209.jpg' wurde nach 'c:\Users\thore\Documents\GitHub\ki\dataset_project\images\train\biological826_jpg.rf.422d96721550d77bd7e8a724d87bd209.jpg' kopiert.
Datei 'biological833_jpg.rf.ebabea1c2368182f58624b5d2847eab6.jpg' wurde nach 'c:\Users\thore\Documents\GitHub\ki\dataset_project\images\train\biological833_jpg.rf

In [3]:
# import os
# import cv2
# import albumentations as A
# from albumentations.pytorch import ToTensorV2
# import numpy as np
# import json
# from tqdm import tqdm

# # Basisordner
# output_path = os.path.join(dir_path, "dataset_project")
# input_path = os.path.join("KI-" + version.version)

# # Eingabeordner
# train_images_folder = os.path.join(output_path, "images", "train")
# annotations_file = os.path.join(train_images_folder, "_annotations.coco.json")

# # Albumentations-Transformationspipeline
# transform = A.Compose(
#     [
#         A.HorizontalFlip(p=0.5),
#         A.RandomRotate90(p=0.5),
#         A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, p=0.7),
#         A.RandomBrightnessContrast(p=0.4),
#         A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.5, p=1.0),  # Kontrast und Helligkeit verändern
#         ToTensorV2()
#     ],
#     bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids'])  # Bounding Box Transformation
# )

# # Anzahl der augmentierten Bilder pro Originalbild
# augmentation_count = 3

# # Lade die Annotationen
# with open(annotations_file, 'r') as f:
#     annotations_data = json.load(f)

# # Erstelle ein Mapping der Bild-ID auf die Annotationen
# image_annotations_map = {}
# for ann in annotations_data['annotations']:
#     if ann['image_id'] not in image_annotations_map:
#         image_annotations_map[ann['image_id']] = []
#     image_annotations_map[ann['image_id']].append(ann)

# # Erstelle ein Mapping der Bild-ID auf den Dateinamen
# image_id_to_filename = {image['id']: image['file_name'] for image in annotations_data['images']}

# # Aktualisiere die zentrale JSON-Datei
# all_annotations = annotations_data

# # Augmentierung der Bilder
# for image_file in tqdm(os.listdir(train_images_folder)):
#     if not image_file.endswith((".jpg", ".png", ".jpeg")):  # Nur Bilddateien bearbeiten
#         continue

#     image_path = os.path.join(train_images_folder, image_file)

#     # Hole die Bild-ID aus der Zuordnung
#     image_id = None
#     for img_id, filename in image_id_to_filename.items():
#         if filename == image_file:
#             image_id = img_id
#             break
#     if image_id is None:
#         print(f"Keine Bild-ID gefunden für Bild: {image_file}")
#         continue

#     # Bild laden
#     image = cv2.imread(image_path)
#     if image is None:
#         print(f"Bild konnte nicht geladen werden: {image_file}")
#         continue

#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Konvertierung von BGR zu RGB

#     # Hole die zugehörigen Annotationen
#     annotations = image_annotations_map.get(image_id, [])

#     # Bounding Boxen und Labels extrahieren
#     bboxes = []
#     category_ids = []
#     for ann in annotations:
#         x, y, w, h = ann['bbox']
#         bboxes.append([x, y, x + w, y + h])  # Umwandlung in [xmin, ymin, xmax, ymax]
#         category_ids.append(ann['category_id'] - 1)  # YOLO Labels beginnen bei 0

#     # Augmentierung durchführen
#     for i in range(augmentation_count):
#         augmented = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#         aug_image = augmented["image"]
#         aug_bboxes = augmented["bboxes"]

#         # Umwandlung des Tensors in ein NumPy-Array
#         aug_image = aug_image.permute(1, 2, 0).numpy()  # Ändert (C, H, W) zu (H, W, C)
#         aug_image = np.clip(aug_image, 0, 255).astype(np.uint8)  # Sicherstellen, dass Werte zwischen 0 und 255 sind

#         # Speichern des augmentierten Bildes
#         aug_image_filename = f"aug_{i}_{image_file}"
#         aug_image_path = os.path.join(train_images_folder, aug_image_filename)
#         aug_image = cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR)  # Zurück zu BGR für OpenCV
#         cv2.imwrite(aug_image_path, aug_image)

#         # Speichern der Annotationen für das augmentierte Bild
#         aug_annotations = []
#         for bbox, category_id in zip(aug_bboxes, category_ids):
#             xmin, ymin, xmax, ymax = bbox
#             w = xmax - xmin
#             h = ymax - ymin
#             aug_annotations.append({
#                 'category_id': category_id + 1,  # Wieder zurück zu originalem Format (1-basierend)
#                 'bbox': [xmin, ymin, w, h],
#                 'image_id': len(all_annotations['images']) + 1,
#                 'id': len(all_annotations['annotations']) + 1
#             })

#         # Füge die augmentierten Annotationen hinzu
#         all_annotations['annotations'].extend(aug_annotations)

#         # Füge die augmentierten Bildinformationen hinzu
#         all_annotations['images'].append({
#             'id': len(all_annotations['images']) + 1,
#             'file_name': aug_image_filename,
#             'height': image.shape[0],
#             'width': image.shape[1]
#         })

# # Überschreibe die zentrale JSON-Datei im selben Ordner
# with open(annotations_file, 'w') as json_file:
#     json.dump(all_annotations, json_file)

# print("Augmentierung abgeschlossen! Alle Dateien und die aktualisierte Annotation befinden sich im selben Ordner.")


ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
 26%|██▌       | 1422/5525 [00:00<00:00, 7063.85it/s]

Keine Bild-ID gefunden für Bild: aug_0_battery765_jpg.rf.31598cadb88068a5951945f4cc4f12f3.jpg
Keine Bild-ID gefunden für Bild: aug_0_battery767_jpg.rf.efc3ac881fbde4439a4059fbd9ec8a04.jpg
Keine Bild-ID gefunden für Bild: aug_0_battery768_jpg.rf.dda9b231ea1a27c4f47af98ad133bd45.jpg
Keine Bild-ID gefunden für Bild: aug_0_battery770_jpg.rf.eb90f6c469fdcd8d0c425cd91aae90d1.jpg
Keine Bild-ID gefunden für Bild: aug_0_battery773_jpg.rf.5e1e5554177547ba2520c38cc5ced9e2.jpg
Keine Bild-ID gefunden für Bild: aug_0_battery77_jpg.rf.3b4ca62d4e57b0d45439df1b4b5ea7a9.jpg
Keine Bild-ID gefunden für Bild: aug_0_battery781_jpg.rf.c1c6cebee6f9b9e1a1cac9711a594dea.jpg
Keine Bild-ID gefunden für Bild: aug_0_battery784_jpg.rf.bb65b4739d409550f914f653114abc3e.jpg
Keine Bild-ID gefunden für Bild: aug_0_battery785_jpg.rf.2c9ee89a3b8f2072f769aec288fde1e6.jpg
Keine Bild-ID gefunden für Bild: aug_0_battery788_jpg.rf.da8eb5ff82d8f8a0a41f7c33ce71abe3.jpg
Keine Bild-ID gefunden für Bild: aug_0_battery78_jpg.rf.b1c4b

 51%|█████     | 2831/5525 [00:00<00:00, 6775.60it/s]

Keine Bild-ID gefunden für Bild: aug_1_battery848_jpg.rf.05c9ae3e1247398d9345b4b3668fe3a5.jpg
Keine Bild-ID gefunden für Bild: aug_1_battery849_jpg.rf.88c0a6da36e9d64e979fa0de205f7dfd.jpg
Keine Bild-ID gefunden für Bild: aug_1_battery850_jpg.rf.a58d5d2edd95c5b681ec8c4522f9afc8.jpg
Keine Bild-ID gefunden für Bild: aug_1_battery851_jpg.rf.14cc83fc7182b8c81148aad23aaf2fcb.jpg
Keine Bild-ID gefunden für Bild: aug_1_battery853_jpg.rf.3b7ff5520afc995654dccbdd5f4651a7.jpg
Keine Bild-ID gefunden für Bild: aug_1_battery856_jpg.rf.dd47fb06d900b04ea4b52065b086b528.jpg
Keine Bild-ID gefunden für Bild: aug_1_battery857_jpg.rf.3f9387d129ff0368fac4e6f5b5fdec25.jpg
Keine Bild-ID gefunden für Bild: aug_1_battery858_jpg.rf.a66c9ef15ce4ec1fdce3fe83ea8e63cb.jpg
Keine Bild-ID gefunden für Bild: aug_1_battery85_jpg.rf.294e741c0100077823ab3b6490c43dd9.jpg
Keine Bild-ID gefunden für Bild: aug_1_battery861_jpg.rf.1428534029b532aea2632074a8ad3b80.jpg
Keine Bild-ID gefunden für Bild: aug_1_battery863_jpg.rf.40cb

 76%|███████▌  | 4201/5525 [00:00<00:00, 6796.14it/s]

Keine Bild-ID gefunden für Bild: aug_2_clothes976_jpg.rf.f678f1e34a77f11e120a4e383879cfd3.jpg
Keine Bild-ID gefunden für Bild: aug_2_clothes977_jpg.rf.6d3cdf16b75a5037180393e627cbb9fc.jpg
Keine Bild-ID gefunden für Bild: aug_2_clothes978_jpg.rf.02337e74de0bfe04742164248cedae78.jpg
Keine Bild-ID gefunden für Bild: aug_2_clothes979_jpg.rf.420d05216d1ff9d3211ccdd9da22ed5e.jpg
Keine Bild-ID gefunden für Bild: aug_2_clothes981_jpg.rf.c610c4c644dd9abd2374ce79e9b34740.jpg
Keine Bild-ID gefunden für Bild: aug_2_clothes982_jpg.rf.4685c255411284bdbd3751a6e7d284b4.jpg
Keine Bild-ID gefunden für Bild: aug_2_clothes983_jpg.rf.82e3e44777f633e225c4b93d85eb4fdb.jpg
Keine Bild-ID gefunden für Bild: aug_2_clothes985_jpg.rf.d8908ea1ec1ec552bc536112bccb7cbf.jpg
Keine Bild-ID gefunden für Bild: aug_2_clothes987_jpg.rf.7afd647482e5636d86d498e84de8c182.jpg
Keine Bild-ID gefunden für Bild: aug_2_clothes989_jpg.rf.b78526cb071f2709c91108ca6b95207d.jpg
Keine Bild-ID gefunden für Bild: aug_2_clothes990_jpg.rf.41c

100%|██████████| 5525/5525 [00:35<00:00, 153.99it/s] 

Augmentierung abgeschlossen! Alle Dateien und die aktualisierte Annotation befinden sich im selben Ordner.


In [55]:
import os
import json

output_path = os.path.join(dir_path, "dataset_project")

# Angenommene Pfade für die JSON-Daten
file_path_train = os.path.join(output_path, "images", "train", "_annotations.coco.json")
file_path_valid = os.path.join(output_path, "images", "valid", "_annotations.coco.json")

# Ausgabeordner für die Labels
output_path = os.path.join(output_path)
train_images_folder = os.path.join(output_path, "images", "train")
valid_images_folder = os.path.join(output_path, "images", "valid")

# Annahme: Die Daten werden aus einer bereits geladenen COCO JSON-Datei extrahiert
with open(file_path_train, 'r') as f:
    data_train = json.load(f)

with open(file_path_valid, 'r') as f:
    data_valid = json.load(f)

# Funktion zum Abrufen der Annotationen für ein Bild
def get_img_ann(image_id, data):
    img_ann = []
    isFound = False
    for ann in data['annotations']:  # Verwendung des entsprechenden Datensatzes
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    return img_ann if isFound else None

# Funktion zum Abrufen der Bildinformationen anhand des Dateinamens
def get_img(filename, data):
    for img in data['images']:  # Verwendung des entsprechenden Datensatzes
        if img['file_name'] == filename:
            return img
    return None

# Verzeichnis für die Label-Dateien erstellen
labels_dir_train = os.path.join(output_path, "labels", "train")
labels_dir_valid = os.path.join(output_path, "labels", "valid")

if not os.path.exists(labels_dir_train):
    os.makedirs(labels_dir_train)  # Sicherstellen, dass der Ordner existiert

if not os.path.exists(labels_dir_valid):
    os.makedirs(labels_dir_valid)  # Sicherstellen, dass der Ordner existiert

# Bilddateinamen aus den Trainings- und Validierungsordnern abrufen
file_names_train = os.listdir(train_images_folder)  # Namen der Trainingsbilder
file_names_valid = os.listdir(valid_images_folder)  # Namen der Validierungsbilder

# Zähler für die Bilder
count = 0

# Funktion, um Label-Dateien zu erstellen
def process_images(file_names, images_folder, labels_dir, data):
    global count
    for filename in file_names:
        print(f"Verarbeite Bild: {filename}")

        # Bildinformationen abrufen
        img = get_img(filename, data)

        if img:
            img_id = img['id']
            img_w = img['width']
            img_h = img['height']

            # Annotationen für dieses Bild abrufen
            img_ann = get_img_ann(img_id, data)

            if img_ann:
                # Label-Datei im YOLO-Format erstellen
                label_file_path = os.path.join(labels_dir, filename.replace('.jpg', '.txt').replace('.png', '.txt'))

                # Verhindern, dass die Datei überschrieben wird, falls sie bereits existiert
                if not os.path.exists(label_file_path):
                    print(f"Erstelle Label-Datei: {label_file_path}")
                    with open(label_file_path, "w") as file_object:  # 'w' überschreibt die Datei
                        for ann in img_ann:
                            current_category = ann['category_id'] - 1  # YOLO-Label beginnen bei 0
                            current_bbox = ann['bbox']
                            x = current_bbox[0]
                            y = current_bbox[1]
                            w = current_bbox[2]
                            h = current_bbox[3]

                            # Berechnung der Mittelpunkte
                            x_centre = (x + (x + w)) / 2
                            y_centre = (y + (y + h)) / 2

                            # Normalisierung der Werte
                            x_centre = x_centre / img_w
                            y_centre = y_centre / img_h
                            w = w / img_w
                            h = h / img_h

                            # Formatieren auf 6 Dezimalstellen
                            x_centre = format(x_centre, '.6f')
                            y_centre = format(y_centre, '.6f')
                            w = format(w, '.6f')
                            h = format(h, '.6f')

                            # Schreiben der Annotation ins Label-File
                            file_object.write(f"{current_category} {x_centre} {y_centre} {w} {h}\n")

                    count += 1  # Zähler für Bildnummer erhöhen
                else:
                    print(f"Label-Datei existiert bereits, überspringe: {label_file_path}")
            else:
                print(f"Keine Annotationen für das Bild: {filename}")
        else:
            print(f"Bild nicht gefunden: {filename}")

# Verarbeite sowohl Trainings- als auch Validierungsbilder
print("Verarbeite Trainingsbilder...")
process_images(file_names_train, train_images_folder, labels_dir_train, data_train)

print("Verarbeite Validierungsbilder...")
process_images(file_names_valid, valid_images_folder, labels_dir_valid, data_valid)

print("Alle Annotationen wurden verarbeitet.")


Verarbeite Trainingsbilder...
Verarbeite Bild: biological775_jpg.rf.1d13aeae9daa6c1048919874d8c1bb5f.jpg
Erstelle Label-Datei: c:\Users\thore\Documents\GitHub\ki\dataset_project\labels\train\biological775_jpg.rf.1d13aeae9daa6c1048919874d8c1bb5f.txt
Verarbeite Bild: biological787_jpg.rf.70df4f651b4a64c037e77101e55ddc55.jpg
Erstelle Label-Datei: c:\Users\thore\Documents\GitHub\ki\dataset_project\labels\train\biological787_jpg.rf.70df4f651b4a64c037e77101e55ddc55.txt
Verarbeite Bild: biological80_jpg.rf.eca050bc55a6feee750cbe400fe9f568.jpg
Erstelle Label-Datei: c:\Users\thore\Documents\GitHub\ki\dataset_project\labels\train\biological80_jpg.rf.eca050bc55a6feee750cbe400fe9f568.txt
Verarbeite Bild: biological826_jpg.rf.422d96721550d77bd7e8a724d87bd209.jpg
Erstelle Label-Datei: c:\Users\thore\Documents\GitHub\ki\dataset_project\labels\train\biological826_jpg.rf.422d96721550d77bd7e8a724d87bd209.txt
Verarbeite Bild: biological833_jpg.rf.ebabea1c2368182f58624b5d2847eab6.jpg
Erstelle Label-Datei:

In [56]:
import json
import os
import yaml

# Verzeichnis für die Ausgabedaten
output_path = os.path.join(dir_path, "dataset_project")
dir_path = os.getcwd()

# Eingabepfade für COCO JSON-Dateien
file_path_train = os.path.join(output_path, "images", "train", "_annotations.coco.json")
file_path_valid = os.path.join(output_path, "images", "valid", "_annotations.coco.json")

# Verzeichnisse für die Trainings- und Validierungsbilder und Labels
yaml_img_train = os.path.join(output_path, "images", "train")
yaml_img_val = os.path.join(output_path, "images", "valid")
yaml_label_train = os.path.join(output_path, "labels", "train")
yaml_label_valid = os.path.join(output_path, "labels", "valid")

# JSON-Dateien laden
with open(file_path_train, 'r') as f:
    data_train = json.load(f)

with open(file_path_valid, 'r') as f:
    data_valid = json.load(f)

# Kategorien extrahieren
category_names = [category['name'] for category in data_train['categories']]
print(category_names)
# Erstellen der YAML-Datenstruktur
yaml_data = {
    "train": './images/train',         # Pfad zu den Trainingsbildern
    "val": './images/valid',           # Pfad zu den Validierungsbildern
    "train_labels": './labels/train',  # Pfad zu den Trainingslabels
    "val_labels": './labels/valid',    # Pfad zu den Validierungslabels
    "nc": len(category_names),        # Anzahl der Kategorien
    "names": category_names,          # Liste der Kategorienamen
    "batch": 8                         # Batch-Größe hinzufügen (z.B. 8)
}

# Pfad zur YAML-Datei
yaml_file_path = os.path.join(output_path, 'annotations.yaml')

# YAML-Datei speichern
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False, allow_unicode=True)

print(f"YAML-Datei wurde erfolgreich gespeichert: {yaml_file_path}")


['battery', 'Biological', 'brown-glass', 'cardboard', 'clothes', 'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'white-glass']
YAML-Datei wurde erfolgreich gespeichert: c:\Users\thore\Documents\GitHub\ki\dataset_project\annotations.yaml


In [6]:
# import torch
# print("CUDA verfügbar:", torch.cuda.is_available())
# print("cuDNN Version:", torch.backends.cudnn.version())
# print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Keine GPU gefunden")
# torch.cuda.device_count()




In [57]:
from ultralytics import YOLO
import requests
import os
import torch
print("CUDA verfügbar:", torch.cuda.is_available())

output_path = os.path.join(dir_path, "dataset_project")
data_path = os.path.join(output_path, "annotations.yaml")

# YOLOv8-Modell laden
model = YOLO("yolov8n.pt")  # Modell: n (Nano), s (Small), m (Medium), etc.

# Training starten
model.train(
    data=(data_path),  # Pfad zur YAML-Datei
    epochs=1,            # Anzahl der Epochen
    imgsz=640,             # Bildgröße (anpassen bei Bedarf)
)

print("Training abgeschlossen!")


CUDA verfügbar: False
New https://pypi.org/project/ultralytics/8.3.63 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.62  Python-3.10.16 torch-2.5.1+cpu CPU (AMD Ryzen 9 3900X 12-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=c:\Users\thore\Documents\GitHub\ki\dataset_project\annotations.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train20, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=Fal

train: Scanning C:\Users\thore\Documents\GitHub\ki\dataset_project\labels\train... 1272 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1272/1272 [00:01<00:00, 731.83it/s]


train: New cache created: C:\Users\thore\Documents\GitHub\ki\dataset_project\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


Argument 'quality_lower' is not valid and will be ignored.
val: Scanning C:\Users\thore\Documents\GitHub\ki\dataset_project\labels\valid... 544 images, 0 backgrounds, 0 corrupt: 100%|██████████| 544/544 [00:00<00:00, 710.91it/s]


val: New cache created: C:\Users\thore\Documents\GitHub\ki\dataset_project\labels\valid.cache
Plotting labels to runs\detect\train20\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train20
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G     0.8883      3.359      1.391         26        640: 100%|██████████| 80/80 [03:26<00:00,  2.58s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:38<00:00,  2.25s/it]


                   all        544        693       0.41      0.242      0.276      0.186

1 epochs completed in 0.069 hours.
Optimizer stripped from runs\detect\train20\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train20\weights\best.pt, 6.2MB

Validating runs\detect\train20\weights\best.pt...
Ultralytics 8.3.62  Python-3.10.16 torch-2.5.1+cpu CPU (AMD Ryzen 9 3900X 12-Core Processor)
Model summary (fused): 168 layers, 3,007,793 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:32<00:00,  1.90s/it]


                   all        544        693      0.409      0.241      0.276      0.185
               battery         42         60      0.419      0.233      0.241      0.128
            Biological         54         74       0.25      0.743      0.428      0.317
           brown-glass         55         60          0          0      0.335      0.247
             cardboard         59         59      0.385     0.0847      0.377       0.22
               clothes         58         79      0.322      0.468      0.273      0.204
           green-glass         60         72      0.229      0.181      0.187      0.134
                 metal         46         46          1          0     0.0908     0.0548
                 paper         57         84      0.453     0.0204     0.0988       0.06
               plastic         54         88      0.224      0.409       0.28      0.174
                 shoes         59         71      0.806      0.268      0.448      0.315
Speed: 1.2ms preproce

In [62]:
# URL des Bildes
image_url = "https://as2.ftcdn.net/jpg/01/37/20/41/1000_F_137204135_2RY3bZ763Dg6uliovXRpOrjhy1n6sNhk.jpg"

# Lokale Datei speichern
local_image_path = "downloaded_image.jpg"
response = requests.get(image_url)

if response.status_code == 200:
    # Bild lokal speichern
    with open(local_image_path, "wb") as file:
        file.write(response.content)
    print(f"Bild erfolgreich heruntergeladen: {local_image_path}")
else:
    print(f"Fehler beim Herunterladen des Bildes: {response.status_code}")

# Trainiertes YOLO-Modell laden
model = YOLO(r"C:\Users\thore\Documents\GitHub\ki\runs\detect\train19\weights\best.pt")

# Inferenz auf dem heruntergeladenen Bild
results = model(local_image_path)

# Ergebnisse anzeigen
# results ist eine Liste von Resultaten, daher durchlaufen wir sie.
for result in results:
    result.show()  # Zeigt das Bild mit erkannten Objekten

# Ergebnisse speichern
for result in results:
    result.save("predictions/")  # Speichert die analysierten Bilder


Bild erfolgreich heruntergeladen: downloaded_image.jpg

image 1/1 c:\Users\thore\Documents\GitHub\ki\downloaded_image.jpg: 448x640 7 Biologicals, 36.0ms
Speed: 1.0ms preprocess, 36.0ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)


In [60]:
import cv2
from ultralytics import YOLO
from pathlib import Path

# Pfad zu deinem trainierten Modell
custom_model_path = r"C:\Users\thore\Documents\GitHub\ki\runs\detect\train19\weights\best.pt"  # Ersetze dies mit dem Pfad zu deiner .pt-Datei
model = YOLO(custom_model_path)  # Lade dein eigenes Modell

# Pfad zum Eingabevideo
video_path = r"C:\Users\thore\Documents\GitHub\ki\demo\WhatsApp Video 2025-01-17 at 13.21.37.mp4"
cap = cv2.VideoCapture(video_path)

# Videoeigenschaften auslesen
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Ausgabevideo konfigurieren
output_path = 'output_video_custom_model.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Objekterkennung im aktuellen Frame mit deinem Modell
    results = model.predict(source=frame, save=False, conf=0.1)  # Konfidenz anpassen, wenn nötig

    # Ergebnisse auf das Bild rendern
    annotated_frame = results[0].plot()

    # Verarbeitetes Bild ins Ausgabevideo schreiben
    out.write(annotated_frame)

    # Optional: Das Bild live anzeigen
    cv2.imshow('Object Detection', annotated_frame)

    # Mit 'q' das Programm vorzeitig beenden
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Ressourcen freigeben
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Objekterkennung abgeschlossen. Video gespeichert unter {output_path}")



0: 640x384 1 cardboard, 37.0ms
Speed: 2.0ms preprocess, 37.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cardboard, 37.0ms
Speed: 1.0ms preprocess, 37.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cardboard, 33.7ms
Speed: 2.0ms preprocess, 33.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cardboard, 36.0ms
Speed: 2.0ms preprocess, 36.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cardboard, 39.5ms
Speed: 2.0ms preprocess, 39.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cardboard, 40.0ms
Speed: 1.0ms preprocess, 40.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cardboard, 38.5ms
Speed: 2.0ms preprocess, 38.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cardboard, 38.0ms
Speed: 2.0ms preprocess, 38.0ms inference, 1.0ms postprocess per 